In [1]:
import numpy as np
import pandas
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix, roc_curve, auc
from sklearn.grid_search import GridSearchCV
from scipy import misc

from imblearn.over_sampling import RandomOverSampler

c:\users\tatiana\appdata\local\programs\python\python35-32\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\tatiana\appdata\local\programs\python\python35-32\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
df = pandas.read_csv('C:\\Tanya\\НИУ ВШЭ\\диплом\\error-recognition-for-R\\rlc_featured_mod2.csv', sep='\t')
df.head()

,sentid,sent,start,finish,error,correction,tag,levenstein,lemmaequal,grammequal,...,par,idiom,coord,refl,not-clear,transp,subst,del,punc,typo
0,61119,Мы сидели пол часа.,3,4,пол часа.,полчаса,space,4,0,0,...,0,0,0,0,0,0,0,0,0,0
1,32,Студенты так же борятся за спасение озера Байк...,2,3,так же,также,orpho,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,107,"(? ). (? ).Мы видим на рынке процесс, по средс...",0,4,NaN,"посредством,",orpho,13,0,0,...,0,0,0,0,0,0,0,0,0,0
3,120,У государства экономические и административные...,3,3,экономические экономические,есть есть экономические,lex,11,0,0,...,0,0,0,0,0,0,0,0,0,0
4,120,У государства экономические и административные...,3,3,экономические экономические,есть есть экономические,lex,11,0,0,...,0,0,0,0,0,0,0,0,0,0


уберем ненужные столбцы

In [3]:
labels = ['levenstein', 'lemmaequal', 'grammequal', 'stemequal', 'lenorig', 'lencorr', 'bastard']
labs = labels[:]
labs.append('lex')
lex_df = df[labs]
lex_df.head()

,levenstein,lemmaequal,grammequal,stemequal,lenorig,lencorr,bastard,lex
0,4,0,0,0,4,1,0,0
1,1,0,0,0,3,2,0,0
2,13,0,0,0,0,2,0,0
3,11,0,0,0,3,4,0,1
4,11,0,0,0,3,4,0,1


In [4]:
lex_df = lex_df.fillna('')

In [5]:
lex_df = pandas.get_dummies(lex_df)
lex_df.head()

,levenstein,lemmaequal,grammequal,stemequal,lenorig,lencorr,bastard,lex
0,4,0,0,0,4,1,0,0
1,1,0,0,0,3,2,0,0
2,13,0,0,0,0,2,0,0
3,11,0,0,0,3,4,0,1
4,11,0,0,0,3,4,0,1


In [6]:
data_train, data_test, answ_train, answ_test = train_test_split(lex_df.drop('lex', axis=1), lex_df['lex'], test_size=0.3)

In [7]:
clf = DecisionTreeClassifier(min_samples_split=5)
gs_clf = GridSearchCV(clf, {'max_leaf_nodes': (-5, -4, -3, -2, -1, None, 2, 3, 4, 5, 6, 7, 8)})
gs_clf.fit(np.array(data_train), np.array(answ_train))
print(gs_clf.best_estimator_)
# for model in gs_clf.grid_scores_:
#     print(model, model[2])
gs_clf.fit(np.array(data_train), np.array(answ_train))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=8, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


GridSearchCV(cv=None, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'max_leaf_nodes': (-5, -4, -3, -2, -1, None, 2, 3, 4, 5, 6, 7, 8)},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=0)

In [8]:
answ_pred = gs_clf.predict(data_test)
print(classification_report(answ_test, answ_pred))

             precision    recall  f1-score   support

          0       0.82      0.98      0.90      4322
          1       0.84      0.27      0.41      1242

avg / total       0.83      0.83      0.79      5564



In [9]:
print(confusion_matrix(answ_test, answ_pred))

[[4256   66]
 [ 906  336]]


модель переобучена на нолики, так как их больше. Нужно или порезать данные, или сделать оверсемплинг?

оверсемплинг

In [10]:
ros = RandomOverSampler()
data_resampled, answ_resampled = ros.fit_sample(lex_df.drop('lex', axis=1), lex_df['lex'])

In [11]:
data_train, data_test, answ_train, answ_test = train_test_split(data_resampled, answ_resampled, test_size=0.3)

In [12]:
clf = DecisionTreeClassifier(min_samples_split=5)
gs_clf = GridSearchCV(clf, {'max_leaf_nodes': (-5, -4, -3, -2, -1, None, 2, 3, 4, 5, 6, 7, 8)})
gs_clf.fit(np.array(data_train), np.array(answ_train))
print(gs_clf.best_estimator_)
# for model in gs_clf.grid_scores_:
#     print(model, model[2])
gs_clf.fit(np.array(data_train), np.array(answ_train))
answ_pred = gs_clf.predict(data_test)
print(classification_report(answ_test, answ_pred))
print(confusion_matrix(answ_test, answ_pred))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=-2, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
             precision    recall  f1-score   support

          0       0.77      0.80      0.78      4349
          1       0.79      0.76      0.77      4321

avg / total       0.78      0.78      0.78      8670

[[3472  877]
 [1047 3274]]


чуть лучше. А если просто безжалостно порезать данные?

In [13]:
ham = lex_df[lex_df.lex == 1].drop('lex', axis=1)
spam = lex_df[lex_df.lex == 0].drop('lex', axis=1)
n = int(len(ham) * 0.7)
print('hey there we train on ', n, 'of ', len(ham), 'while spam length is', len(spam))
hams = ham.sample(n)
spams = spam.sample(n)

hey there we train on  2867 of  4096 while spam length is 14449


In [14]:
for index, row in ham.iterrows():
    #print(row)
    pass
# ham_test = [i for index, i in ham.iterrows() if not hams.isin(i)]
ham_test = ham - hams
spam_test = spam - spams
print(len(hams), len(spams), len(ham_test), len(spam_test))



2867 2867 4096 14449


In [15]:
train = pandas.concat([hams, spams])
test = ham_test + spam_test
train_answers = [1 for i in hams] + [0 for i in spams]
test_answers = [1 for i in ham_test] + [0 for i in spam_test]
print(train.head())
print(len(train), len(train_answers), len(test), len(test_answers))

       levenstein  lemmaequal  grammequal  stemequal  lenorig  lencorr  \
754            59           0           0          0        8        3   
2550            5           0           0          0        1        1   
15481           3           0           0          0        1        2   
17995           2           0           0          0        1        1   
4571            8           0           0          0        1        2   

       bastard  
754          0  
2550         1  
15481        0  
17995        1  
4571         0  
5734 14 18545 14


In [85]:
clf = DecisionTreeClassifier(min_samples_split=5)
gs_clf = GridSearchCV(clf, {'max_leaf_nodes': (-5, -4, -3, -2, -1, None, 2, 3, 4, 5, 6, 7, 8)})
gs_clf.fit(np.array(data_train), np.array(answ_train))
print(gs_clf.best_estimator_)
# for model in gs_clf.grid_scores_:
#     print(model, model[2])
gs_clf.fit(np.array(train), np.array(train_answers))
answ_pred = gs_clf.predict(test)
print(classification_report(test_answers, answ_pred))
print(confusion_matrix(test_answers, answ_pred))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=-5, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=5,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')


ValueError: Found input variables with inconsistent numbers of samples: [7276, 14]